<a href="https://colab.research.google.com/github/202311003/idm_assignment_3/blob/202311003/CP_3_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [3]:
deliveries = pd.read_csv('/content/drive/MyDrive/CP3/deliveries.csv')
matches = pd.read_csv('/content/drive/MyDrive/CP3/matches.csv')
points_table = pd.read_csv('/content/drive/MyDrive/CP3/points_table.csv')
stats2023 = pd.read_csv('/content/drive/MyDrive/CP3/2023stats.csv')
# stats2023 = pd.read_csv('/content/drive/MyDrive/CP3/wc_2023_stats.csv')
wc_stats2023 = pd.read_csv('/content/drive/MyDrive/CP3/wc_2023_stats.csv')

In [4]:
stats2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Player Name     150 non-null    object
 1   Runs            150 non-null    int64 
 2   Fours           150 non-null    int64 
 3   Sixes           150 non-null    int64 
 4   Wickets         150 non-null    int64 
 5   Balls Bowled    150 non-null    int64 
 6   Bowls faced     150 non-null    int64 
 7   Matches played  150 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 9.5+ KB


In [5]:
stats2023['Batting Strike Rate'] = ((stats2023['Runs'] / stats2023['Bowls faced']) * 100).round(2)
stats2023['Bowling Strike Rate'] = ((stats2023['Balls Bowled'] / stats2023['Wickets'])).round(2)

wc_stats2023['Batting Strike Rate'] = ((stats2023['Runs'] / stats2023['Bowls faced']) * 100).round(2)
wc_stats2023['Bowling Strike Rate'] = ((stats2023['Balls Bowled'] / stats2023['Wickets'])).round(2)

In [39]:
stats2023['Batting Avg'] = ((stats2023['Runs'] / stats2023['Matches played'])).round(2)
wc_stats2023['Batting Avg'] = ((wc_stats2023['Runs'] / wc_stats2023['Matches played'])).round(2)

In [41]:
for x in wc_stats2023['Batting Avg']:
  print(x)

2.11
2.71
10.44
19.33
62.78
27.11
18.14
11.67
12.14
17.4
11.5
10.0
8.12
17.71
22.62
2.88
15.88
7.43
16.33
6.5
3.75
3.33
2.25
17.38
16.2
23.4
8.17
3.5
6.0
26.57
21.5
42.29
0.33
5.67
2.0
11.67
6.11
39.22
8.67
3.11
40.0
17.5
3.0
4.0
1.38
19.62
1.88
6.11
5.0
2.12
0.0
2.75
0.0
1.12
13.88
8.88
0.5
14.25
56.71
21.0
6.0
8.0
0.17
15.5
5.33
12.5
36.0
0.0
35.57
17.4
2.67
52.25
27.0
2.0
2.67
16.5
7.5
0.0
2.0
12.6
18.5
1.0
6.5
0.0
10.2
44.0
2.33
6.0
1.5
22.12
2.33
0.0
0.0
40.0
5.0
39.0
35.56
19.5
46.62
27.8
13.88
39.89
34.33
73.0
27.0
35.25
51.29
35.33
13.5
18.33
30.25
31.11
41.78
34.44
3.5
13.62
31.0
22.62
65.67
20.71
49.11
36.22
28.33
36.89
21.29
32.67
41.44
55.75
29.29
35.75
0.0
55.25
23.5
67.88
36.62
35.0
22.14
16.4
36.43
18.0
48.0
18.71
62.33
36.5
29.67
48.5
44.0
21.25
26.33
7.5


In [42]:
stats2023.replace([np.inf, -np.inf], np.nan, inplace=True)
stats2023 = stats2023.fillna(0)

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [44]:
X = stats2023[['Matches played','Batting Strike Rate','Batting Avg']]
# X = stats2023[['Runs', 'Fours', 'Sixes', 'Wickets', 'Balls Bowled', 'Balls Faced', 'Matches Played','Batting Strike Rate','Bowling Strike Rate']]
y_fours = stats2023['Fours']
y_sixes = stats2023['Sixes']
players = stats2023['Player Name']

In [45]:
X_train_fours, X_test_fours, y_fours_train, y_fours_test, players_train_fours, players_test_fours = train_test_split(
    X, y_fours, players, test_size=0.2, random_state=42
)

In [46]:
X_train_sixes, X_test_sixes, y_sixes_train, y_sixes_test, players_train_sixes, players_test_sixes = train_test_split(
    X, y_sixes, players, test_size=0.2, random_state=42
)


In [47]:
scaler = StandardScaler()
X_train_fours_scaled = scaler.fit_transform(X_train_fours)
X_test_fours_scaled = scaler.transform(X_test_fours)

In [48]:
X_train_sixes_scaled = scaler.fit_transform(X_train_sixes)
X_test_sixes_scaled = scaler.transform(X_test_sixes)

In [49]:
def create_model(layers=3, units=64, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(Dense(units, input_dim=X_train_fours_scaled.shape[1], activation=activation))

    for _ in range(layers - 1):
        model.add(Dense(units, activation=activation))

    model.add(Dense(1, activation='linear'))

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [50]:
class CustomKerasRegressor:
    def __init__(self, layers=3, units=64, activation='relu', optimizer='adam'):
        self.layers = layers
        self.units = units
        self.activation = activation
        self.optimizer = optimizer
        self.model = None

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

    def fit(self, X, y):
        self.model = create_model(layers=self.layers, units=self.units, activation=self.activation, optimizer=self.optimizer)
        self.model.fit(X, y, epochs=50, batch_size=32, verbose=0)
        return self

    def predict(self, X):
        return self.model.predict(X).flatten()

In [56]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', CustomKerasRegressor())
])

# Define hyperparameters for grid search
param_grid = {
    'regressor__layers': [2, 3, 4],
    'regressor__units': [32, 64, 128],
    'regressor__activation': ['relu', 'tanh'],
    'regressor__optimizer': ['adam', 'rmsprop'],
    'regressor__dropout_rate': [0.0, 0.2, 0.5],
    'regressor__epochs': [50, 100, 150],
}

# Perform grid search
grid = GridSearchCV(pipeline, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_result = grid.fit(X_train_fours_scaled, y_fours_train)

# Get the best model from grid search
best_model = grid_result.best_estimator_

print(best_model)
# Make predictions
y_fours_pred = best_model.predict(X_test_fours_scaled)

# Display predictions for fours
predictions_fours_df = pd.DataFrame({
    'Player Name': players_test_fours,
    'Predicted Fours': y_fours_pred.round().astype(int),
})

# Display predictions for fours
print(predictions_fours_df)

# Evaluate the best model for fours
mse_fours = mean_squared_error(y_fours_test, y_fours_pred)
print(f'Mean Squared Error for Fours: {mse_fours}')

2/2 [==============================] - 0s 15ms/step
Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor',
                 <__main__.CustomKerasRegressor object at 0x78d127ebff70>)])
1/1 [==============================] - 0s 104ms/step
            Player Name  Predicted Fours
73         CBRLS Kumara                1
18      Vikramjit Singh                7
118           Q de Kock               57
78        MADI Hemantha                3
76         Hasan Mahmud                1
31          Mahmudullah               31
64           RJW Topley                6
141           JP Inglis               12
68             MR Marsh               27
82           Noor Ahmad                4
110          SA Edwards               26
12               A Dutt                6
36        Mohammad Nabi                7
9                MM Ali                6
19       Saqib Zulfiqar                1
56         JR Hazlewood                1
104         Imam-ul-Haq               16
69  

In [57]:
# Load and preprocess the test dataset
# X_test_wc = wc_stats2023.drop(columns=['Fours'])  # Adjust columns as needed
# X_test_wc = wc_stats2023[['Runs', 'Sixes', 'Wickets', 'Balls Bowled', 'Bowls faced', 'Matches played','Batting Strike Rate','Bowling Strike Rate']]

X_test_wc = wc_stats2023[['Matches played','Batting Strike Rate','Batting Avg']]

y_fours_test_wc = wc_stats2023['Fours']  # Assuming 'Fours' is the column for your target variable in the test dataset

wc_stats2023.replace([np.inf, -np.inf], np.nan, inplace=True)
wc_stats2023 = wc_stats2023.fillna(0)

X_test_wc_scaled = scaler.transform(X_test_wc)
y_fours_pred_wc = best_model.predict(X_test_wc_scaled)

predictions_wc_df = pd.DataFrame({
    'Player Name': wc_stats2023['Player Name'],  # Replace with the actual column name in your test dataset
    'Predicted Fours': y_fours_pred_wc.round().astype(int),
})

print(predictions_wc_df)

mse_fours = mean_squared_error(y_fours_test_wc, y_fours_pred_wc)
print(f'Mean Squared Error for Fours: {mse_fours}')

5/5 [==============================] - 0s 3ms/step
         Player Name  Predicted Fours
0           TA Boult                3
1           MJ Henry                2
2         MJ Santner                8
3        JDS Neesham                7
4         R Ravindra               55
..               ...              ...
145     RR Hendricks               10
146        BA Stokes               20
147         SA Yadav                9
148        W Barresi                6
149  FDM Karunaratne                2

[150 rows x 2 columns]
Mean Squared Error for Fours: 13.787952748315165


In [60]:
predictions_wc_df.sort_values(by='Predicted Fours', ascending=False).head(15)

,Player Name,Predicted Fours
118,Q de Kock,57
4,R Ravindra,55
133,V Kohli,52
131,RG Sharma,46
127,DA Warner,46
71,DJ Mitchell,44
120,HE van der Dussen,44
58,GJ Maxwell,42
85,AK Markram,42
98,DJ Malan,39
